# Margin Minimization (CVXPY edition)

Refer to `allocation.ipynb` to follow along.

In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd
from scipy import sparse

π = pd.read_csv("data/portfolio.csv", index_col=0)
M = len(π)

mv = (π["Shares"] * π["Price"]).to_numpy()
bounds = [np.minimum(mv, 0), np.maximum(mv, 0)]
sign = np.sign(π["Shares"]).to_numpy()
keys, inverse = np.unique(π["Sector"], return_inverse=True)
K = len(keys)
sectors = sparse.csr_array((np.ones(M), (inverse, np.arange(M, dtype=int))), shape=(K, M))

In [ ]:
x0 = cp.Variable(shape=M, name="x₀", bounds=bounds)
x1 = cp.Variable(shape=M, name="x₁", bounds=bounds)
abs_x0 = cp.multiply(sign, x0)
gmv = cp.sum(abs_x0)

objective = cp.Minimize(
    0.05 * gmv
    + 0.2 * cp.sum(cp.pos(abs_x0 - π["Volume"] * π["Price"]))
    + 0.1 * cp.sum(cp.pos(abs_x0 - 0.05 * gmv))
    + 0.2 * cp.sum(cp.pos(cp.abs(sectors @ x0) - 0.2 * gmv))
    + 0.06 * cp.sum(cp.multiply(sign, x1))
)
constraints = [x0 + x1 == mv]

problem = cp.Problem(objective, constraints)
data, chain, inverse_data = problem.get_problem_data(solver=cp.CLARABEL)
soln = chain.solve_via_data(problem, data)
problem.unpack_results(soln, chain, inverse_data)

π.assign(**{"Account 0": x0.value, "Account 1": x1.value})

Now confirm that [CVXPY](https://www.cvxpy.org/) generated the constraints we expect:

In [ ]:
def same(m: sparse.dia_array, n: sparse.dia_array):
    m = m.todia()
    return np.array_equal(m.offsets, n.offsets) and np.allclose(m.data, n.data)

# x = [x₀, liquidity, concentration, sector, x₁, |x₀ by sector|]

A = data["A"]
one = sparse.eye_array(M)
rows = slice(None, M)
assert same(A[rows, :-(M+K)], one) and same(A[:M, -(M+K):], sparse.eye_array(M)), "x₀ + x₁ == x"

rows = slice(M, 2 * M)
assert same(A[rows, :], -one), "x₀ ≥ x₋"

rows = slice(2 * M, 3 * M)
assert same(A[rows, :], one), "x₀ ≤ x₊"

rows = slice(3 * M, 4 * M)
assert not A[rows, :-(M + K)].nnz and same(A[rows, -(M + K):], -one), "x₁ ≥ x₋"

rows = slice(4 * M, 5 * M)
assert not A[rows, :-(M + K)].nnz and same(A[rows, -(M + K):], one), "x₁ ≤ x₊"

rows = slice(5 * M, 6 * M)
assert same(A[rows, :M], sparse.diags_array(sign)) and same(A[rows, M:], -one),  "x₀ - liquidity ≤ ADV"

rows = slice(6 * M, 7 * M)
assert not A[rows, :M].nnz and same(A[rows, M:], -one),  "liquidity ≥ 0"

rows = slice(7 * M, 8 * M)
assert (
    np.allclose(A[rows, :M].todense(), np.diag(sign) - 0.05 * sign[np.newaxis, :])
    and not A[rows, M: 2 * M].nnz
    and same(A[rows, 2 * M:], -one)
), "x₀ - 5% G - concentration ≤ 0"

rows = slice(8 * M, 9 * M)
assert not A[rows, : 2 * M].nnz and same(A[rows, 2 * M:], -one),  "concentration ≥ 0"

one = sparse.eye_array(K)
rows = slice(9 * M, 9 * M + K)
assert (
    np.allclose(A[rows, :M].todense(), sectors.todense())
    and not A[rows, M: -K].nnz
    and same(A[rows, -K:], -one)
), "x₀ by sector - |x₀ by sector| ≤ 0"

rows = slice(9 * M + K, 9 * M + 2 * K)
assert (
    np.allclose(A[rows, :M].todense(), -sectors.todense())
    and not A[rows, M: -K].nnz
    and same(A[rows, -K:], -one)
), "-x₀ by sectors - |x₀ by sector| ≤ 0"

rows = slice(9 * M + 2 * K, 9 * M + 3 * K)
assert (
    np.allclose(A[rows, :M].todense(), -0.2 * sign[np.newaxis, :])
    and not A[rows, M: -(M + 2 * K)].nnz
    and same(A[rows, -(M + 2 * K): -K], -one)
    and same(A[rows, -K:], one)
), "-20% G - sector + |x₀ by sector| ≤ 0"

rows = slice(9 * M + 3 * K, None)
assert not A[rows, :3 * M].nnz and same(A[rows, 3 * M:], -one), "sector ≥ 0"